# The Battle of Neighborhoods *part 2*
## Lewis Tian 

## Task 1: Scrape the table and get the required dataframe

In [67]:
import pandas as pd
import numpy as np
from pandas.io.html import read_html

### Web scraping using pandas

In [24]:
src='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table=read_html(src,attrs={'class':'wikitable'})
print('Table extracted!')

Table extracted!


In [25]:
#Print some info of the result it scraped
print('There are {} table(s) in this page'.format(len(table)))
print('This table varible is a {} object'.format(type(table)))
print('This table in it is a {} object'.format(type(table[0])))

There are 1 table(s) in this page
This table varible is a <class 'list'> object
This table in it is a <class 'pandas.core.frame.DataFrame'> object


In [124]:
# Look at the data
df = table[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### Clean the dataframe

In [125]:
df = df[df['Borough']!='Not assigned']
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


After cleaning the Neighborhoods that have unassigned borough, let's see how many of them have <b>neighborhood as 'Not assigned'</b>

In [126]:
df[df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,Not assigned


It turns out that there is only one Neighbourhood that needs to be fixed which is shown above

In [127]:
#Manually change the neighbourhood name
df['Neighbourhood'][9] = 'Queen\'s Park'

Now we see that there is no neighborhood that is not assigned as shown below

In [128]:
df[df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


Use groupby function to group neighborhoods together and join them 

In [129]:
df=df.groupby(["Postcode",'Borough'])['Neighbourhood'].apply(','.join).reset_index()

Now we are done with creating and cleaning the dataframe! Let's see the head its shape.

In [130]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [131]:
df.shape

(103, 3)

## Task 2: Get geographical coordinates and expand the previous table

#### Read the geographical coordinates from given file

Since the coordinates are already sorted by its postcode, they cna be appended directly to the existing dataframe

In [132]:
coor=pd.read_csv('Geospatial_Coordinates.csv')
df['Latitude']=coor['Latitude']
df['Longitude']=coor['Longitude']

In [133]:
#check some info and see its shape
print(df.shape)
df.head(10)

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## Task 3: Cluster the neighborhoods

In [162]:
#first import libs that are needed for further visulization and analysis
import folium
import requests
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Since exploring all the neighborhoods might take a while to run, only boroughs containing <b>Toronto</b> are selected for further analysis

In [134]:
df = df[df['Borough'].str.contains("Toronto", case=False)]
print(df.shape)
df.head()

(39, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Get some more info about the processed dataframe

In [137]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [155]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [156]:
## Lets visualise the neighborhood using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [157]:
CLIENT_ID = 'X14P2GYHG42D0SWXLLHDZWDDDS3BCI3XMGRNVOW4RYF0G2EJ' # your Foursquare ID
CLIENT_SECRET = 'DL1R2LFTLA2RBVZURFRGDKWKRSGJFUWBLIPVMCUPYSFM4Y45' # your Foursquare Secret
VERSION = '20200205' # Foursquare API version
radius = 1000
LIMIT = 200
print('Ready to use Foursquare API!')

Ready to use Foursquare API!


#### Define a function to help get nearby venues

In [206]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Explore the venues of our target neighborhoods

In [207]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [208]:
#take a peek of the venues we get
print(toronto_venues.shape)
toronto_venues.head()

(1720, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
3,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [209]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


### Analyze neighborhoods

In [210]:
# One hot encoding for categories
toronto_one_hot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# Add neighborhood columns back
toronto_one_hot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
# move neighborhood column to the first column
toronto_one_hot = toronto_one_hot.set_index('Neighbourhood').reset_index()

toronto_one_hot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [211]:
toronto_one_hot.shape

(1720, 241)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [212]:
toronto_grouped = toronto_one_hot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,...,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [213]:
#check the new size
toronto_grouped.shape

(39, 241)

In [214]:
# Lets print each neighboorhood with the top 5 most common venue
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.06
1              Café  0.04
2               Bar  0.04
3   Thai Restaurant  0.04
4  Sushi Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1            Beer Bar  0.04
2      Farmers Market  0.04
3   French Restaurant  0.04
4  Seafood Restaurant  0.04


----Brockton,Exhibition Place,Parkdale Village----
                   venue  freq
0                   Café  0.14
1         Breakfast Spot  0.09
2            Coffee Shop  0.09
3  Performing Arts Venue  0.05
4                Stadium  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0                 Spa  0.07
1             Butcher  0.07
2       Garden Center  0.07
3  Light Rail Station  0.07
4      Farmers Market  0.07


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service

#### putting the result above to dataframe

In [215]:
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

convert the top venues to dataframe

In [216]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Thai Restaurant,Sushi Restaurant,Cosmetics Shop,Restaurant,Burger Joint,Steakhouse,Bakery
1,Berczy Park,Coffee Shop,Cheese Shop,Steakhouse,Bakery,Seafood Restaurant,French Restaurant,Farmers Market,Café,Beer Bar,Cocktail Bar
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Gym,Intersection,Performing Arts Venue,Pet Store,Grocery Store,Nightclub,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Spa,Auto Workshop,Pizza Place,Restaurant,Butcher,Burrito Place,Skate Park,Brewery,Farmers Market,Comic Shop
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Plane,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Harbor / Marina


### Cluster neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [217]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 1, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 4, 0, 0])

In [218]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Health Food Store,Other Great Outdoors,Neighborhood,Pub,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Furniture / Home Store,Ice Cream Shop,Brewery,Bubble Tea Shop,Café,Caribbean Restaurant
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Pizza Place,Sandwich Place,Liquor Store,Ice Cream Shop,Pet Store,Coffee Shop,Pub,Movie Theater,Burrito Place,Burger Joint
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,Brewery,Gastropub,American Restaurant,Bar,Clothing Store,Stationery Store
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Photography Studio,Construction & Landscaping,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Breakfast Spot,Department Store,Park,Hotel,Sandwich Place,Food & Drink Shop,Donut Shop,Discount Store,Dog Run
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Gym / Fitness Center,Shoe Store,Salon / Barbershop,Restaurant,Pet Store,Park,Mexican Restaurant
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Pizza Place,Dessert Shop,Sushi Restaurant,Italian Restaurant,Gym,Coffee Shop,Café,Indoor Play Area,Flower Shop
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,1,Tennis Court,Playground,Park,Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,0,Coffee Shop,Pub,Pizza Place,Fried Chicken Joint,Restaurant,Liquor Store,Sports Bar,Supermarket,Sushi Restaurant,Light Rail Station


Finally, let's visualize the clusters

In [221]:
#import additional modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [223]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [225]:
#cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Furniture / Home Store,Ice Cream Shop,Brewery,Bubble Tea Shop,Café,Caribbean Restaurant
42,East Toronto,0,Pizza Place,Sandwich Place,Liquor Store,Ice Cream Shop,Pet Store,Coffee Shop,Pub,Movie Theater,Burrito Place,Burger Joint
43,East Toronto,0,Café,Coffee Shop,Italian Restaurant,Bakery,Brewery,Gastropub,American Restaurant,Bar,Clothing Store,Stationery Store
45,Central Toronto,0,Gym,Breakfast Spot,Department Store,Park,Hotel,Sandwich Place,Food & Drink Shop,Donut Shop,Discount Store,Dog Run
46,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Gym / Fitness Center,Shoe Store,Salon / Barbershop,Restaurant,Pet Store,Park,Mexican Restaurant
47,Central Toronto,0,Sandwich Place,Pizza Place,Dessert Shop,Sushi Restaurant,Italian Restaurant,Gym,Coffee Shop,Café,Indoor Play Area,Flower Shop
49,Central Toronto,0,Coffee Shop,Pub,Pizza Place,Fried Chicken Joint,Restaurant,Liquor Store,Sports Bar,Supermarket,Sushi Restaurant,Light Rail Station
51,Downtown Toronto,0,Café,Restaurant,Coffee Shop,Park,Italian Restaurant,Pub,Bakery,Pizza Place,Pet Store,Outdoor Sculpture
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Gym,Hotel,Dance Studio
53,Downtown Toronto,0,Coffee Shop,Bakery,Park,Pub,Theater,Café,Breakfast Spot,Restaurant,Mexican Restaurant,Bank


Cluster 1 seems to contain mostly restaurants

In [226]:
#cluster2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Tennis Court,Playground,Park,Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant
50,Downtown Toronto,1,Park,Playground,Trail,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Cluster 2 seems to be recreation places

In [227]:
#cluster3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,2,Photography Studio,Construction & Landscaping,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Cluster 3 seems to be quite mixed with different types of venues

In [228]:
#cluster4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Pool,Garden,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Cluster 4 seems to be very mixed too

In [229]:
#cluster5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,4,Trail,Health Food Store,Other Great Outdoors,Neighborhood,Pub,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


Finally, Cluster 5 seems to contain food venues mostly!

# Thanks for reading this notebook! Feel free to provide insights or feedbacks that can improve this.